In [1]:
import pandas as pd
import numpy as np
import random
import sys
import os
import time
import codecs
import collections
from six.moves import cPickle

from __future__ import print_function
#import Keras library
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import categorical_accuracy

#import spacy, and spacy french model
# spacy is used to work on text
import spacy
import spacy
import fr_core_news_md

nlp = fr_core_news_md.load()

#define parameters used in the tutorial
save_dir = 'save' # directory to store trained NN models

In [2]:
### In order to get 'cah-cards-full.json' you need to download 'Download full.json'
### on the following website: https://www.crhallberg.com/cah/

df = pd.read_json("cah-cards-full.json")
df = df.T

white_sentences = np.array([])
for index, row in df.iterrows(): 
  try:
    length = len(row['white'][index]['text'])
      
    for i in row['white']:
      white_sentences = np.append(white_sentences, i['text'])

  except:
    print("List index out of range")

black_sentences = np.array([])
for index, row in df.iterrows(): 
  try:
    length = len(row['black'][index]['text'])

    for i in row['black']:
      # So we only keep the sentence with one proposition
      if i['pick'] > 1:
        print("Pick more than 1")  
      else: 
        black_sentences = np.append(black_sentences, i['text'])

  except:
    print("List index out of range")

List index out of range
List index out of range
List index out of range
List index out of range
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pick more than 1
Pic

In [3]:
print("Number white sentences :"+str(len(white_sentences)))
print("Number black sentences :"+str(len(black_sentences)))


Number white sentences :13327
Number black sentences :2838


In [4]:
def create_wordlist(doc):
    wl = []
    for word in doc:
        if word.text not in ("\n","\n\n",'\u2009','\xa0', '#', '%', '®', '/', '-', '.', 'a'):
            wl.append(word.text.lower())
    return wl

In [5]:
doc = ' '.join(white_sentences)
doc = nlp(doc)
wl = create_wordlist(doc)
wordlist = wl


In [6]:
# count the number of words
word_counts = collections.Counter(wordlist)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

#size of the vocabulary
vocab_size = len(words)
print("vocab size: ", vocab_size)


#save the words and vocabulary
# vocab_file = os.path.join(save_dir, "words_vocab.pkl")
# with open(os.path.join(vocab_file), 'wb') as f:
#     cPickle.dump((words, vocab, vocabulary_inv), f)

vocab size:  13377


In [7]:
#create sequences
sequences = []
next_words = []
seq_length = 100
sequences_step = 10
for i in range(0, len(wordlist) - seq_length, sequences_step):
    sequences.append(wordlist[i: i + seq_length])
    next_words.append(wordlist[i + seq_length])

print('nb sequences:', len(sequences))

nb sequences: 6439


In [12]:
# Vocab size is too large for Colab so need to set a limit
limit = 6000
vocab_limit = dict(list(vocab.items())[:len(vocab)-limit])
X = np.zeros((len(sequences), seq_length, vocab_size - limit), dtype=np.bool)
y = np.zeros((len(sequences), vocab_size - limit), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, word in enumerate(sentence):
      try:
        X[i, t, vocab_limit[word]] = 1
      except:
        print("No X index")
    try:
      y[i, vocab_limit[next_words[i]]] = 1
    except:
      print("No Y index")
    

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X index
No X inde

In [13]:
def bidirectional_lstm_model(seq_length, vocab_size):
    print('Build LSTM model.')
    model = Sequential()
    model.add(Bidirectional(LSTM(rnn_size, activation="relu"),input_shape=(seq_length, vocab_size)))
    model.add(Dropout(0.6))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    
    optimizer = Adam(lr=learning_rate)
    callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
    print("model built!")
    return model

In [14]:
rnn_size = 256 # size of RNN
seq_length = 30 # sequence length
learning_rate = 0.001 #learning rate

md = bidirectional_lstm_model(seq_length, vocab_size)
md.summary()

Build LSTM model.
model built!
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 512)               27922432  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 13377)             6862401   
_________________________________________________________________
activation_1 (Activation)    (None, 13377)             0         
Total params: 34,784,833
Trainable params: 34,784,833
Non-trainable params: 0
_________________________________________________________________


In [17]:
batch_size = 32 # minibatch size
num_epochs = 50 # number of epochs

callbacks=[EarlyStopping(patience=4, monitor='val_loss')]
#fit the model
history = md.fit(X, y,
                 batch_size=batch_size,
                 shuffle=True,
                 epochs=num_epochs,
                 callbacks=callbacks,
                 validation_split=0.1)

#save the model
md.save()

Epoch 1/50


ValueError: ignored